# This is a notebook for classification according to Wolfram.

In [13]:
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import time

wolfram_labels = {
    0: 'I', 8: 'I', 32: 'I', 40: 'I', 128: 'I', 136: 'I', 160: 'I', 168: 'I',
    1: 'II', 2: 'II', 3: 'II', 4: 'II', 5: 'II', 6: 'II', 7: 'II', 9: 'II',
    10: 'II', 11: 'II', 12: 'II', 13: 'II', 14: 'II', 15: 'II', 19: 'II', 23: 'II',
    24: 'II', 25: 'II', 26: 'II', 27: 'II', 28: 'II', 29: 'II', 33: 'II', 34: 'II',
    35: 'II', 36: 'II', 37: 'II', 38: 'II', 42: 'II', 43: 'II', 44: 'II', 46: 'II',
    50: 'II', 51: 'II', 56: 'II', 57: 'II', 58: 'II', 62: 'II', 72: 'II', 73: 'II',
    74: 'II', 76: 'II', 77: 'II', 78: 'II', 94: 'II', 104: 'II', 108: 'II',
    130: 'II', 132: 'II', 134: 'II', 138: 'II', 140: 'II', 142: 'II', 152: 'II',
    154: 'II', 156: 'II', 162: 'II', 164: 'II', 170: 'II', 172: 'II', 178: 'II',
    184: 'II', 200: 'II', 204: 'II', 232: 'II',
    18: 'III', 22: 'III', 30: 'III', 45: 'III', 60: 'III', 90: 'III', 105: 'III',
    122: 'III', 126: 'III', 146: 'III', 150: 'III',
    41: 'IV', 54: 'IV', 106: 'IV', 110: 'IV'
}

CLASS_MAP = {'I': 0, 'II': 1, 'III': 2, 'IV': 3}

# you can use these default values or change them as needed
N_STEPS = 1000
N_CELLS = 100

In [15]:
### ------ ECA Evolution Function ------ ###
"""
In here, you input the rule, steps and cells you want to evolve.
"""
def evolve_eca(rule, n_steps=N_STEPS, n_cells=N_CELLS):
    rule_binary = format(rule, '08b')[::-1]
    lookup = {format(i, '03b'): int(rule_binary[i]) for i in range(8)}
    
    current = np.zeros(n_cells, dtype=int)
    current[n_cells//2] = 1
    space_time = np.zeros((n_steps, n_cells), dtype=int)
    space_time[0] = current
    
    for t in range(1, n_steps):
        new_state = np.zeros(n_cells, dtype=int)
        for i in range(n_cells):
            neighborhood = f"{current[(i-1)%n_cells]}{current[i]}{current[(i+1)%n_cells]}"
            new_state[i] = lookup[neighborhood]
        current = new_state
        space_time[t] = current
    
    return space_time

In [16]:
### ------ Feature Extraction ------ ###
def lempel_ziv_complexity(sequence):
    s = ''.join(map(str, sequence.flatten()))
    n = len(s)
    i, k, l = 0, 1, 1
    c, k_max = 1, 1
    
    while i + k <= n:
        temp = s[i:i+k]
        if i > 0 and temp in s[i-l:i+k-1]:
            k += 1
        else:
            c += 1
            i += k
            k = 1
            k_max = max(k_max, i+k-l)
            l = k_max
    
    return c / (n / np.log2(n + 1e-10))

def extract_features(space_time):
    n_steps, n_cells = space_time.shape
    features = {}
    
    rule_density = np.mean(space_time)
    features['lambda'] = rule_density
    
    flat = space_time.flatten()
    p = np.bincount(flat, minlength=2) / len(flat)
    features['shannon_entropy'] = -np.sum(p * np.log2(p + 1e-10))
    
    features['lz_complexity'] = lempel_ziv_complexity(space_time)
    
    hamming_distances = [np.sum(space_time[t] != space_time[t-1]) / n_cells 
                        for t in range(1, n_steps)]
    features['hamming_mean'] = np.mean(hamming_distances)
    features['hamming_std'] = np.std(hamming_distances)
    
    active_widths = []
    for t in range(n_steps):
        if np.any(space_time[t] == 1):
            active_indices = np.where(space_time[t] == 1)[0]
            active_widths.append(active_indices[-1] - active_indices[0])
    features['spreading_rate'] = np.polyfit(range(len(active_widths)), 
                                            active_widths, 1)[0] if len(active_widths) > 10 else 0
    
    stability_threshold = 0.05
    stable_time = n_steps
    for t in range(10, n_steps):
        recent_changes = [np.sum(space_time[t-i] != space_time[t-i-1]) / n_cells 
                         for i in range(min(10, t))]
        if np.mean(recent_changes) < stability_threshold:
            stable_time = t
            break
    features['time_to_stability'] = stable_time / n_steps
    
    features['activity'] = rule_density
    
    fft_peaks = []
    for col in range(min(10, n_cells)):
        if np.std(space_time[:, col]) > 0:
            fft = np.abs(np.fft.rfft(space_time[:, col]))
            peaks, _ = signal.find_peaks(fft, height=np.max(fft)*0.1)
            fft_peaks.append(len(peaks))
    features['periodicity_score'] = np.mean(fft_peaks) if fft_peaks else 0
    
    change_rates = [np.sum(space_time[t] != space_time[t-1]) / n_cells 
                   for t in range(1, n_steps)]
    features['chaos_indicator'] = np.std(change_rates)
    
    return features

In [17]:
### ------ Batch Processing ------ ###
def extract_all_features(rules):
    all_features = []
    computation_times = []
    
    for rule in rules:
        start_time = time.time()
        space_time = evolve_eca(rule)
        features = extract_features(space_time)
        features['rule'] = rule
        features['known_class'] = wolfram_labels[rule]
        all_features.append(features)
        computation_times.append(time.time() - start_time)
    
    df = pd.DataFrame(all_features)
    df['computation_time'] = computation_times
    
    return df

In [ ]:
### ------ Classification ------ ###
def classify_rules(df):
    feature_cols = ['lambda', 'shannon_entropy', 'lz_complexity', 'hamming_mean', 
                   'hamming_std', 'spreading_rate', 'time_to_stability', 
                   'activity', 'periodicity_score', 'chaos_indicator']
    
    X = df[feature_cols].values
    y = df['known_class'].map(CLASS_MAP).values
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', 
                                 random_state=726, max_depth=10)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    predictions = np.zeros(len(y))
    
    for train_idx, test_idx in skf.split(X_scaled, y):
        clf.fit(X_scaled[train_idx], y[train_idx])
        predictions[test_idx] = clf.predict(X_scaled[test_idx])
    
    df['predicted_class_num'] = predictions.astype(int)
    inv_map = {v: k for k, v in CLASS_MAP.items()}
    df['predicted_class'] = df['predicted_class_num'].map(inv_map)
    
    return df, clf, scaler

In [19]:
### ------ Results ------ ###
def print_results(df):
    print("Classification Report:")
    y_true = df['known_class'].map(CLASS_MAP)
    y_pred = df['predicted_class'].map(CLASS_MAP)
    print(classification_report(y_true, y_pred, 
                                target_names=['I', 'II', 'III', 'IV']))
    
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=['I', 'II', 'III', 'IV'], 
                        columns=['I', 'II', 'III', 'IV'])
    print(cm_df)
    
    print(f"\nTotal computation time: {df['computation_time'].sum():.2f}s")
    print(f"Average time per rule: {df['computation_time'].mean():.3f}s")
    
    return df[['rule', 'known_class', 'predicted_class', 'computation_time']]

In [20]:
rules = list(wolfram_labels.keys())
df_features = extract_all_features(rules)
df_results, model, scaler = classify_rules(df_features)
results_summary = print_results(df_results)

Classification Report:
              precision    recall  f1-score   support

           I       0.67      1.00      0.80         8
          II       0.90      0.85      0.87        65
         III       0.47      0.64      0.54        11
          IV       0.00      0.00      0.00         4

    accuracy                           0.80        88
   macro avg       0.51      0.62      0.55        88
weighted avg       0.78      0.80      0.78        88


Confusion Matrix:
     I  II  III  IV
I    8   0    0   0
II   4  55    6   0
III  0   4    7   0
IV   0   2    2   0

Total computation time: 328.67s
Average time per rule: 3.735s
